In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm_notebook

%matplotlib inline

PATH = '../'

In [2]:
def smape(satellite_predicted_values, satellite_true_values): 
    return np.mean(np.abs(satellite_predicted_values - satellite_true_values) / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values)))

In [3]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

Wall time: 2.54 s


In [4]:
import plyades
import astropy
from astropy import units as units

In [5]:
submissionTrain = submission.iloc[0:1, :].copy()
submissionTrain.drop(index = 0, inplace = True)

In [26]:
trainSize = 0.5
smp = []
submissionTrain = submission.iloc[0:1, :].copy()
submissionTrain.drop(index = 0, inplace = True)
for index, ID in tqdm_notebook(enumerate(test['sat_id'].unique())):
    if index >= 0:
        dataTrain = train[train['sat_id'] == ID]
        size = int(dataTrain.shape[0] * trainSize)
        dataTest = dataTrain.iloc[size:]
        dataTrain = dataTrain.iloc[:size]

        dt = dataTrain['epoch'].iloc[-1]
        vec = dataTrain.iloc[-1, 3:9]
        xx,yy,zz,vx,vy,vz = vec[0], vec[1], vec[2], vec[3], vec[4], vec[5]
        iss_r = np.array([xx,yy,zz,]) * astropy.units.km
        iss_v = np.array([vx,vy,vz,]) * astropy.units.km/astropy.units.s
        iss_t = astropy.time.Time(dt)
        frame = 'ECI'
        body = plyades.bodies.EARTH
        iss = plyades.State(iss_r, iss_v, iss_t, frame, body)

        @property
        def elements(self):
            return kepler.elements(self.body.mu, self.r, self.v)

        @iss.gravity
        def newton_j2(f, t, y, params):
            r = np.sqrt(np.square(y[:3]).sum())
            mu = params['body'].mu.value
            j2 = params['body'].j2
            r_m = params['body'].mean_radius.value
            rx, ry, rz = y[:3]
            f[:3] += y[3:]
            pj = -3/2*mu*j2*r_m**2/r**5
            f[3] += -mu*rx/r**3 + pj*rx*(1-5*rz**2/r**2)
            f[4] += -mu*ry/r**3 + pj*ry*(1-5*rz**2/r**2)
            f[5] += -mu*rz/r**3 + pj*rz*(3-5*rz**2/r**2)

            
        delta = (dataTest['epoch'].apply(lambda x: pd.to_datetime(x)).iloc[-1] - dataTest['epoch'].apply(lambda x: pd.to_datetime(x)).iloc[-2]) 
        nextDate = 'T'.join(str(dataTrain['epoch'].apply(lambda x: pd.to_datetime(x)).iloc[-1] + delta).split(' '))
        frac = (pd.to_datetime(dataTest['epoch']).iloc[-1] + delta - pd.to_datetime(nextDate)) / pd.Timedelta('365 days 7 hours 12 minutes')
        j2_orbit = iss.propagate(dt = frac * units.year, max_step =300, interpolate=dataTest.shape[0] + 1)
        predictions = pd.DataFrame(np.asarray(j2_orbit.table['rx', 'ry', 'rz', 'vx', 'vy', 'vz'])).values[1:]
        
        real = dataTest[['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
        submissionTrain = submissionTrain.append(dataTest[submissionTrain.columns])
        submissionTrain.loc[dataTest.index, 1:] = predictions 
        smp.append(100*(1 - smape(real, predictions)))
        print(ID, smp[-1])

1 91.59655562007882
2 99.03499112781712
3 98.96840687074308
4 91.4284945500594
6 87.00807039827157
9 97.65970334483909
16 98.45482309648189
20 85.7313292512599
22 85.84422298546231
24 99.49005515968999
25 92.68862047128202
26 84.1833158320685
27 90.63212530514157
28 90.6642051777305
29 99.18234014117581
32 98.2880257333698
34 98.75693941637344
35 76.36064714031049
36 95.96917566674827
37 84.33766490694558
38 91.50906973786338
39 98.02778498074362
40 98.51010907041331
41 88.76248840184148
42 97.43638501459499
44 95.00306771237875
45 90.5163591587552
49 97.36305592811998
51 97.82373244718353
52 90.24946175675542
53 96.1071232951128
54 88.48337538478282
57 97.46681513939554
59 91.27957695190268
63 89.00805272613479
64 97.92549356571256
68 95.3873831759217
70 97.77789593738947
75 94.98007730123221
82 83.17465132456896
84 91.79160101384534
86 86.56343871978027
89 97.88793304709338
90 91.07289410445608
91 92.76095773345246
92 89.76428716246257
93 97.99780343461005
96 98.32275006653188
98 89.

In [28]:
submissionTrain = submissionTrain.reset_index()
submissionTrain.to_csv('submTrain.csv', index = None)

In [394]:
for index, ID in tqdm_notebook(enumerate(test['sat_id'].unique())):
    if index == 5:
        dataTrain = train[train['sat_id'] == ID]
        dataTest = test[test['sat_id'] == ID]

        dt = dataTrain['epoch'].iloc[-1]
        vec = dataTrain.iloc[-1, 3:9]
        xx,yy,zz,vx,vy,vz = vec[0], vec[1], vec[2], vec[3], vec[4], vec[5]
        iss_r = np.array([xx,yy,zz,]) * astropy.units.km
        iss_v = np.array([vx,vy,vz,]) * astropy.units.km/astropy.units.s
        iss_t = astropy.time.Time(dt)
        frame = 'ECI'
        body = plyades.bodies.EARTH
        iss = plyades.State(iss_r, iss_v, iss_t, frame, body)

        @property
        def elements(self):
            return kepler.elements(self.body.mu, self.r, self.v)

        @iss.gravity
        def newton_j2(f, t, y, params):
            r = np.sqrt(np.square(y[:3]).sum())
            mu = params['body'].mu.value
            j2 = params['body'].j2
            r_m = params['body'].mean_radius.value
            rx, ry, rz = y[:3]
            f[:3] += y[3:]
            pj = -3/2*mu*j2*r_m**2/r**5
            f[3] += -mu*rx/r**3 + pj*rx*(1-5*rz**2/r**2)
            f[4] += -mu*ry/r**3 + pj*ry*(1-5*rz**2/r**2)
            f[5] += -mu*rz/r**3 + pj*rz*(3-5*rz**2/r**2)

        delta = (dataTest['epoch'].apply(lambda x: pd.to_datetime(x)).iloc[-1] - dataTest['epoch'].apply(lambda x: pd.to_datetime(x)).iloc[-2]) 
        nextDate = 'T'.join(str(dataTrain['epoch'].apply(lambda x: pd.to_datetime(x)).iloc[-1] + delta).split(' '))
        frac = (pd.to_datetime(dataTest['epoch']).iloc[-1] + delta - pd.to_datetime(nextDate)) / pd.Timedelta('365 days 7 hours 12 minutes')
        j2_orbit = iss.propagate(dt = frac * units.year, max_step =300, interpolate=dataTest.shape[0] + 1)
        predictions = pd.DataFrame(np.asarray(j2_orbit.table['rx', 'ry', 'rz', 'vx', 'vy', 'vz'])).values[1:]
        
        submission.loc[dataTest.index, 1:] = predictions

In [397]:
j2_orbit.table

dt,epoch,rx,ry,rz,vx,vy,vz,semi_major_axis,eccentricity,inclination,ascending_node,argument_of_periapsis,true_anomaly
s,,km,km,km,km / s,km / s,km / s,km,,rad,rad,rad,rad
float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.0,2014-01-31T22:30:01.579,58377.9316948366,-65285.629254606414,15318.69744984667,0.6444044810738814,1.490834716373179,-0.9152636104988768,72592.90265813173,0.4882600574387648,0.5162665164478747,2.6136434055634155,1.041387607490307,2.4567447896210948
8123.512810763242,2014-02-01T00:45:25.092,62393.25322414302,-51937.693917440825,7625.650251109176,0.32693610331461875,1.7970194928748269,-0.974618170712498,72593.01861209406,0.48826084477450454,0.5162670308158464,2.613643105262668,1.041390123415325,2.2908087595893316
16247.025621526484,2014-02-01T03:00:48.605,63345.73458369722,-36081.0467001067,-421.70056984503793,-0.11846148732918649,2.1055310928213,-0.9985325104257301,72593.20014647019,0.48826259202548283,0.5162672632567012,2.6136430454122404,1.0413930609195012,2.0884812371965604
24370.538432289726,2014-02-01T05:16:12.117,59939.022452467936,-17814.71270131554,-8404.10646159801,-0.7607344440593863,2.379993204603822,-0.9494540471746118,72593.47687680261,0.4882660466279857,0.5162667021866122,2.6136428332821238,1.0413945916218597,1.8269718182015602
32494.051243052967,2014-02-01T07:31:35.630,50177.9856536881,2186.018904705325,-15419.924907973216,-1.7007804322151243,2.498790246086039,-0.7388949853823781,72593.78666326575,0.48827162680693265,0.5162641757917933,2.613640150670936,1.0413873545848178,1.4634933676345956
40617.56405381621,2014-02-01T09:46:59.143,31384.795968118768,21407.709780829377,-19470.9241097858,-2.960825254271597,2.0939346512414154,-0.1800493750737065,72593.72627502131,0.4882757823322803,0.516258745742793,2.6136252183893536,1.0413553740664203,0.9218819783964922
48741.07686457945,2014-02-01T12:02:22.656,2915.2759430179976,33024.380804781154,-17025.67268512338,-3.8598361623835435,0.5603234181717461,0.8290545281754523,72594.26752660352,0.4882815144142732,0.5162588785472966,2.6135859284968377,1.0413015060290174,0.14092802195524745
56864.58967534269,2014-02-01T14:17:46.169,-26665.30031105786,29317.475929219807,-6748.789634584477,-3.162765989816822,-1.3642403520526618,1.5734004799984391,72595.85772844242,0.48828977019945063,0.5162688789150047,2.6135637391727133,1.0412289319350103,0.6942256302958043


In [398]:
dataTest

,id,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
4059,22783,9,2014-02-01T00:45:26.204,64447.774403,-57051.075814,8069.726248,0.444755,1.631005,-0.937650
4060,22784,9,2014-02-01T03:00:50.829,66610.109924,-42636.673709,328.994191,0.068595,1.915114,-0.960795
4061,22785,9,2014-02-01T05:16:15.453,65193.903627,-25974.719919,-7408.591579,-0.444660,2.178774,-0.931520
4062,22786,9,2014-02-01T07:31:40.078,58845.579441,-7427.715839,-14546.612180,-1.157303,2.363666,-0.801950
4063,22787,9,2014-02-01T09:47:04.703,45667.411909,11809.524923,-19911.983291,-2.131419,2.306382,-0.472703
4064,22788,9,2014-02-01T12:02:29.328,23758.336780,28365.434706,-21262.447822,-3.245940,1.617432,0.209916
4065,22789,9,2014-02-01T14:17:53.953,-5299.307496,35335.227449,-15698.960252,-3.688285,-0.033216,1.156629
4066,22790,9,2014-02-01T16:33:18.578,-32197.732421,27882.850832,-3726.034611,-2.755874,-1.666682,1.669877
4067,22791,9,2014-02-01T18:48:43.203,-49145.553172,11041.321276,9777.885761,-1.445588,-2.330762,1.590620
4068,22792,9,2014-02-01T21:04:07.827,-56634.535248,-8312.942533,21590.554260,-0.461047,-2.372109,1.306501


In [382]:
submission.to_csv('submission.csv', index = None)